In [ ]:
import pyodbc;

server = 'ews.database.windows.net'
database = 'ews_assets'
username = 'ews_admin'
password = 'Test1234!' 
driver = '{ODBC Driver 18 for SQL Server}' 
connection_string = f'DRIVER={driver};SERVER={server};PORT=1433;DATABASE={database};UID={username};PWD={password}'
print(f"Connection String: {connection_string}")


In [ ]:
conn = pyodbc.connect(connection_string)
print("Connection successful!")

In [ ]:
cursor = conn.cursor()
cursor.execute("SELECT TOP 10000 * FROM EWS")
for row in cursor.fetchall():
        print(row)

In [ ]:
cursor.close()
conn.close()